# 多向量检索器

每个文档存储多个向量通常是有益的。在许多用例中，这是有益的。 LangChain 有一个基础 MultiVectorRetriever ，这使得查询此类设置变得容易。很多复杂性在于如何为每个文档创建多个向量。本笔记本涵盖了创建这些向量和使用 MultiVectorRetriever 的一些常见方法。

为每个文档创建多个向量的方法包括：
- 较小的块：将文档分割成较小的块，然后嵌入这些块（这是 ParentDocumentRetriever）。
- 摘要：为每个文档创建摘要，将其与文档一起嵌入（或代替文档）
- 假设性问题：创建每个文档都适合回答的假设性问题，将这些问题与文档一起嵌入（或代替文档）。

请注意，这还启用了另一种添加嵌入的方法 - 手动。这很棒，因为您可以显式添加导致文档恢复的问题或查询，从而为您提供更多控制权。

In [1]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore

loaders = [
    TextLoader("/mnt/f/kk_datasets/txt/faq-4359.txt"),
    TextLoader("/mnt/f/kk_datasets/txt/faq-7923.txt"),
    TextLoader("/mnt/f/kk_datasets/txt/yjhx.md"),
    TextLoader("/mnt/f/kk_datasets/txt/agent.md"),
]

loaders

In [2]:
docs = []
for loader in loaders:
    docs.extend(loader.load())

docs

[Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-4359.txt'}, page_content='一、什么是0分期利息\n\n您好，“0分期利息”是指买家使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息的功能，分期利息由华为商城承担。\n\n注：自2023年起，商城将相关宣传将“免息”调整为“0分期利息”，主要基于中国银保监会、中国人民银行《关于进一步促进信用卡业务规范健康发展的通知》（银保监规〔2022〕13号），要求“银行业金融机构应当在分期业务合同（协议）首页和业务办理页面以明显方式展示分期业务可能产生的所有息费项目、年化利率水平和息费计算方式。向客户展示分期业务收取的资金使用成本时，应当统一采用利息形式，并明确相应的计息规则，不得采用手续费等形式，法律法规另有规定的除外。”\n\n\n\n二、可以参与0分期利息活动的商品\n\n商城目前仅支持部分单品参与0分期利息，若多商品（含不支持0分期利息）合并支付则不支持0分期利息，以支付页面为准，后续会逐渐开放更多商品，敬请关注。\n\n\n\n三、确认订单分期成功\n\n订单提交成功后在支付方式页面选择分期支付，点选显示0分期利息的支付方式及具体0分期利息期数后，完成支付。\n\n\n\n\n\n四、订单中有多个商品，其中有商品支持0分期利息，为什么提交后却没有0分期利息？\n\n0分期利息商品不能和其它商品一起购买，如果和其他商品购买而不能享用0分期利息，建议取消原来的订单，重新购买时把0分期利息商品和其他商品分开单独购买；且0分期利息的分期数是固定的，如6期0分期利息，并不是3/6/12都提供0分期利息的。\n\n\n\n五、小程序是否支持0分期利息？\n\n华为商城小程序暂不支持0分期利息。'),
 Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-7923.txt'}, page_content='众测活动\n\n整体介绍：\n\n一、活动定义：众测是以低价试销的形式，通过收集评价、销量等方法，用于测试市场对新商品的反应，用于及时优化销售策略和引导商家改进。\n\n二、优势：众测价通常比较优惠，以不高于大促促销价为原则，最终以与物权方谈判结果为准

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

docs = text_splitter.split_documents(docs)

docs


[Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-4359.txt'}, page_content='一、什么是0分期利息\n\n您好，“0分期利息”是指买家使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息的功能，分期利息由华为商城承担。\n\n注：自2023年起，商城将相关宣传将“免息”调整为“0分期利息”，主要基于中国银保监会、中国人民银行《关于进一步促进信用卡业务规范健康发展的通知》（银保监规〔2022〕13号），要求“银行业金融机构应当在分期业务合同（协议）首页和业务办理页面以明显方式展示分期业务可能产生的所有息费项目、年化利率水平和息费计算方式。向客户展示分期业务收取的资金使用成本时，应当统一采用利息形式，并明确相应的计息规则，不得采用手续费等形式，法律法规另有规定的除外。”\n\n\n\n二、可以参与0分期利息活动的商品\n\n商城目前仅支持部分单品参与0分期利息，若多商品（含不支持0分期利息）合并支付则不支持0分期利息，以支付页面为准，后续会逐渐开放更多商品，敬请关注。\n\n\n\n三、确认订单分期成功\n\n订单提交成功后在支付方式页面选择分期支付，点选显示0分期利息的支付方式及具体0分期利息期数后，完成支付。\n\n\n\n\n\n四、订单中有多个商品，其中有商品支持0分期利息，为什么提交后却没有0分期利息？\n\n0分期利息商品不能和其它商品一起购买，如果和其他商品购买而不能享用0分期利息，建议取消原来的订单，重新购买时把0分期利息商品和其他商品分开单独购买；且0分期利息的分期数是固定的，如6期0分期利息，并不是3/6/12都提供0分期利息的。\n\n\n\n五、小程序是否支持0分期利息？\n\n华为商城小程序暂不支持0分期利息。'),
 Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-7923.txt'}, page_content='众测活动\n\n整体介绍：\n\n一、活动定义：众测是以低价试销的形式，通过收集评价、销量等方法，用于测试市场对新商品的反应，用于及时优化销售策略和引导商家改进。\n\n二、优势：众测价通常比较优惠，以不高于大促促销价为原则，最终以与物权方谈判结果为准

In [4]:
# 导入本地词嵌入模型
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="/home/libing/kk_LLMs/bge-large-zh-v1.5")


/tmp/ipykernel_1177/1980583364.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="/home/libing/kk_LLMs/bge-large-zh-v1.5")
/home/libing/miniconda3/envs/kk_langchain_wsl/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
# 用于索引子块的向量存储
vectorstore = Chroma(collection_name="full_documents", embedding_function=embeddings)


/tmp/ipykernel_1177/3936686920.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(collection_name="full_documents", embedding_function=embeddings)


In [6]:
# 父文档的存储层
store = InMemoryStore()
id_key = "doc_id"

# 检索器（空启动）
retriever = MultiVectorRetriever(vectorstore=vectorstore, byte_store=store, id_key=id_key)

In [7]:
import uuid

doc_ids = [str(uuid.uuid4()) for _ in docs]

doc_ids


['932ce001-7423-4b73-a323-0667eedc75cb',
 '6f7cf9ed-7747-4465-ae5c-daa750d4d1e2',
 '3efdb766-d459-4cee-9ba6-97694da19920',
 'ae3b6144-2a7f-44d7-8ffa-be67d57bb7f1',
 'd6aef0e6-4017-47f5-a28d-b120001d6d13',
 '37369a86-0a1e-4b86-ae4b-bb9b241d380b',
 '5c0a7805-d70f-4534-a9ff-9d0d257f7852']

In [8]:
# 创建较小块的分割器
from langchain_text_splitters import CharacterTextSplitter

child_text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=10, length_function=len, separator="\n\n", is_separator_regex=False)

In [9]:
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _doc = child_text_splitter.split_documents([doc])
    for sub_doc in _doc:
        sub_doc.metadata[id_key] = _id
    sub_docs.extend(_doc)

sub_docs

Created a chunk of size 224, which is longer than the specified 100
Created a chunk of size 117, which is longer than the specified 100
Created a chunk of size 176, which is longer than the specified 100
Created a chunk of size 120, which is longer than the specified 100
Created a chunk of size 319, which is longer than the specified 100
Created a chunk of size 123, which is longer than the specified 100
Created a chunk of size 135, which is longer than the specified 100
Created a chunk of size 115, which is longer than the specified 100
Created a chunk of size 119, which is longer than the specified 100
Created a chunk of size 147, which is longer than the specified 100
Created a chunk of size 124, which is longer than the specified 100
Created a chunk of size 104, which is longer than the specified 100
Created a chunk of size 103, which is longer than the specified 100
Created a chunk of size 163, which is longer than the specified 100
Created a chunk of size 108, which is longer tha

[Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-4359.txt', 'doc_id': '932ce001-7423-4b73-a323-0667eedc75cb'}, page_content='一、什么是0分期利息\n\n您好，“0分期利息”是指买家使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息的功能，分期利息由华为商城承担。'),
 Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-4359.txt', 'doc_id': '932ce001-7423-4b73-a323-0667eedc75cb'}, page_content='注：自2023年起，商城将相关宣传将“免息”调整为“0分期利息”，主要基于中国银保监会、中国人民银行《关于进一步促进信用卡业务规范健康发展的通知》（银保监规〔2022〕13号），要求“银行业金融机构应当在分期业务合同（协议）首页和业务办理页面以明显方式展示分期业务可能产生的所有息费项目、年化利率水平和息费计算方式。向客户展示分期业务收取的资金使用成本时，应当统一采用利息形式，并明确相应的计息规则，不得采用手续费等形式，法律法规另有规定的除外。”'),
 Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-4359.txt', 'doc_id': '932ce001-7423-4b73-a323-0667eedc75cb'}, page_content='二、可以参与0分期利息活动的商品\n\n商城目前仅支持部分单品参与0分期利息，若多商品（含不支持0分期利息）合并支付则不支持0分期利息，以支付页面为准，后续会逐渐开放更多商品，敬请关注。'),
 Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-4359.txt', 'doc_id': '932ce001-7423-4b73-a323-0667eedc75cb'}, page_content='三、确认订单分期成功\n\n订单提交成功后在支付方式页面选择分期支付，点选显示0分期利息的支付方式及具体

In [10]:
# 使用一个名为retriever的对象来向一个向量存储vectorstore添加文档
# 并且使用一个文档存储docstore来设置文档ID和文档内容的映射
# 这两个属性分别用于存储文档的向量化表示和文档的内容

retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, sub_docs)))

In [11]:
# vectorstore 单独检索小块

retriever.vectorstore.similarity_search("什么是LangGraph？")

[Document(metadata={'doc_id': 'd6aef0e6-4017-47f5-a28d-b120001d6d13', 'source': '/mnt/f/kk_datasets/txt/agent.md'}, page_content='LangGraph 是一种基于语言模型与图谱结合的技术，旨在通过将自然语言处理（NLP）与知识图谱的优势结合，提升信息检索、语义理解和推荐系统的能力。它通过引入图谱结构，将文本数据转换为图形结构，增强了语义关联性，并能在更复杂的情境下推理和推荐。'),
 Document(metadata={'doc_id': 'd6aef0e6-4017-47f5-a28d-b120001d6d13', 'source': '/mnt/f/kk_datasets/txt/agent.md'}, page_content='LangGraph 的核心思想是将传统的知识图谱和深度学习中的预训练语言模型（如 GPT、BERT 等）相结合。传统的知识图谱是通过节点（实体）和边（关系）构成的网络，主要用于表示实体之间的知识关系。而语言模型则能够理解并生成自然语言，在一定程度上捕捉语言的语法和语义。'),
 Document(metadata={'doc_id': 'd6aef0e6-4017-47f5-a28d-b120001d6d13', 'source': '/mnt/f/kk_datasets/txt/agent.md'}, page_content='LangGraph 主要用于以下几个方面：\n\n- 提高知识图谱的构建效率和准确性\n- 实现智能推荐和个性化搜索\n- 增强语义搜索的精度和效率\n- 支持基于图谱的推理和决策'),
 Document(metadata={'doc_id': 'd6aef0e6-4017-47f5-a28d-b120001d6d13', 'source': '/mnt/f/kk_datasets/txt/agent.md'}, page_content='LangGraph 通过以下两种方式将两者结合：')]

## 摘要总结

通常，摘要可能能够更准确地提炼出某个块的内容，从而实现更好的检索。在这里，我们展示如何创建摘要，然后嵌入它们。

In [12]:
import uuid

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("LOCAL_API_KEY")
base_url = os.getenv("LOCAL_API_BASE")

model = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3, max_tokens=8192)

In [13]:
# 创建链

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("总结下面的文档: \n\n{doc}")
    | model
    | StrOutputParser()
)



In [14]:
docs = []

for loader in loaders:
    docs.extend(loader.load())

docs


[Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-4359.txt'}, page_content='一、什么是0分期利息\n\n您好，“0分期利息”是指买家使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息的功能，分期利息由华为商城承担。\n\n注：自2023年起，商城将相关宣传将“免息”调整为“0分期利息”，主要基于中国银保监会、中国人民银行《关于进一步促进信用卡业务规范健康发展的通知》（银保监规〔2022〕13号），要求“银行业金融机构应当在分期业务合同（协议）首页和业务办理页面以明显方式展示分期业务可能产生的所有息费项目、年化利率水平和息费计算方式。向客户展示分期业务收取的资金使用成本时，应当统一采用利息形式，并明确相应的计息规则，不得采用手续费等形式，法律法规另有规定的除外。”\n\n\n\n二、可以参与0分期利息活动的商品\n\n商城目前仅支持部分单品参与0分期利息，若多商品（含不支持0分期利息）合并支付则不支持0分期利息，以支付页面为准，后续会逐渐开放更多商品，敬请关注。\n\n\n\n三、确认订单分期成功\n\n订单提交成功后在支付方式页面选择分期支付，点选显示0分期利息的支付方式及具体0分期利息期数后，完成支付。\n\n\n\n\n\n四、订单中有多个商品，其中有商品支持0分期利息，为什么提交后却没有0分期利息？\n\n0分期利息商品不能和其它商品一起购买，如果和其他商品购买而不能享用0分期利息，建议取消原来的订单，重新购买时把0分期利息商品和其他商品分开单独购买；且0分期利息的分期数是固定的，如6期0分期利息，并不是3/6/12都提供0分期利息的。\n\n\n\n五、小程序是否支持0分期利息？\n\n华为商城小程序暂不支持0分期利息。'),
 Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-7923.txt'}, page_content='众测活动\n\n整体介绍：\n\n一、活动定义：众测是以低价试销的形式，通过收集评价、销量等方法，用于测试市场对新商品的反应，用于及时优化销售策略和引导商家改进。\n\n二、优势：众测价通常比较优惠，以不高于大促促销价为原则，最终以与物权方谈判结果为准

In [15]:
summaries = chain.batch(docs, {"max_concurrency": 5})

summaries


['本文档介绍了“0分期利息”这一购物支付方式的概念及其使用规则。“0分期利息”指的是消费者在使用花呗等分期付款工具购买商品时，无需承担分期利息，该利息由华为商城承担。自2023年起，为了响应监管机构的要求，宣传用语从“免息”调整为“0分期利息”。文档还指出，目前仅部分单品支持此功能，并且不支持与非0分期利息商品合并支付；此外，消费者需要在订单提交时选择相应的分期支付方式来享受该优惠。最后提到华为商城小程序暂未提供此项服务。',
 '文档总结了华为商城开展的众测活动的相关信息：\n\n- **活动定义**：通过低价试销收集市场反馈以优化销售策略和改进商品。\n- **优势与适用范围**：价格优惠，适用于华为商城所有产品。\n\n**参与方式**：\n- 通过搜索“众测”进入入口；\n- 上新频次不定期，每周一至周五更新；\n- 每款产品的测试时间一般为10到20天，热销商品可能延长5到10天。\n\n**常见问题解答**包括了关于众测商品的来源、价格优惠情况、质量保证、发货时间、订单支付时效、订单查询方法、取消订单及退款流程以及参与次数限制等信息。总体而言，华为商城通过众测活动为消费者提供了接触新品的机会，并以优惠政策吸引用户参与测试和反馈。',
 '本文档介绍了在VMALL平台上进行设备回收的具体步骤，包括选择要回收的物品、价格评估、查看估价结果以及提交订单的过程。\n\n一、用户首先需要通过点击VMALL网站上的“以旧换新”标志进入相应的页面，在这里可以选择想要回收的商品类别和品牌。如果找不到自己的品牌或型号，则可以通过点击加号来查找所有支持的品牌和型号。\n\n二、选择好商品后，系统会引导用户进行价格评估。在这个过程中，用户需要提供关于设备的详细信息，包括网络制式、存储容量等基本信息以及外观状况等，并最终通过“免费询价”按钮提交这些数据以获取估价结果。\n\n三、在收到估价反馈之后，如果对结果满意，则可以继续下一步操作；如果有任何疑问或希望调整评估参数，可以选择重新询价或者更换机型。确认无误后点击“换钱”，进入订单页面完成回收流程的最后一步。\n\n四、提交订单时，用户可以从两种不同的回收方式中选择一种：顺丰快递上门取货或是质检师直接到访进行检测和回收。每种方法都有其特定的操作步骤，包括提供必要的个人信息以及预约合适的时间等。\n\n五、对于在使用过程中遇到的

In [16]:
# 将摘要添加到向量存储中
vectorstore = Chroma(collection_name="summaries", embedding_function=embeddings)
store = InMemoryStore()
id_key = "doc_id"
retriever = MultiVectorRetriever(vectorstore=vectorstore, byte_store=store, id_key=id_key)

doc_ids = [str(uuid.uuid4()) for _ in docs]


In [17]:
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]}) for i, s in enumerate(summaries)
]

summary_docs


[Document(metadata={'doc_id': '0a8c93ce-9cae-46ee-a679-e2dda18101dd'}, page_content='本文档介绍了“0分期利息”这一购物支付方式的概念及其使用规则。“0分期利息”指的是消费者在使用花呗等分期付款工具购买商品时，无需承担分期利息，该利息由华为商城承担。自2023年起，为了响应监管机构的要求，宣传用语从“免息”调整为“0分期利息”。文档还指出，目前仅部分单品支持此功能，并且不支持与非0分期利息商品合并支付；此外，消费者需要在订单提交时选择相应的分期支付方式来享受该优惠。最后提到华为商城小程序暂未提供此项服务。'),
 Document(metadata={'doc_id': '5754f1ab-66e1-44ea-a13f-8729a9fcf654'}, page_content='文档总结了华为商城开展的众测活动的相关信息：\n\n- **活动定义**：通过低价试销收集市场反馈以优化销售策略和改进商品。\n- **优势与适用范围**：价格优惠，适用于华为商城所有产品。\n\n**参与方式**：\n- 通过搜索“众测”进入入口；\n- 上新频次不定期，每周一至周五更新；\n- 每款产品的测试时间一般为10到20天，热销商品可能延长5到10天。\n\n**常见问题解答**包括了关于众测商品的来源、价格优惠情况、质量保证、发货时间、订单支付时效、订单查询方法、取消订单及退款流程以及参与次数限制等信息。总体而言，华为商城通过众测活动为消费者提供了接触新品的机会，并以优惠政策吸引用户参与测试和反馈。'),
 Document(metadata={'doc_id': '5affe8c7-3a40-4a4c-bc07-da76132c0c45'}, page_content='本文档介绍了在VMALL平台上进行设备回收的具体步骤，包括选择要回收的物品、价格评估、查看估价结果以及提交订单的过程。\n\n一、用户首先需要通过点击VMALL网站上的“以旧换新”标志进入相应的页面，在这里可以选择想要回收的商品类别和品牌。如果找不到自己的品牌或型号，则可以通过点击加号来查找所有支持的品牌和型号。\n\n二、选择好商品后，系统会引导用户进行价格评估。在这个过程中，用户需要提供关于设备的详细信息，包括网络制式、存

In [18]:
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))


In [19]:
sub_docs = retriever.vectorstore.similarity_search("众测活动是否有参与限制?")

sub_docs



[Document(metadata={'doc_id': '5754f1ab-66e1-44ea-a13f-8729a9fcf654'}, page_content='文档总结了华为商城开展的众测活动的相关信息：\n\n- **活动定义**：通过低价试销收集市场反馈以优化销售策略和改进商品。\n- **优势与适用范围**：价格优惠，适用于华为商城所有产品。\n\n**参与方式**：\n- 通过搜索“众测”进入入口；\n- 上新频次不定期，每周一至周五更新；\n- 每款产品的测试时间一般为10到20天，热销商品可能延长5到10天。\n\n**常见问题解答**包括了关于众测商品的来源、价格优惠情况、质量保证、发货时间、订单支付时效、订单查询方法、取消订单及退款流程以及参与次数限制等信息。总体而言，华为商城通过众测活动为消费者提供了接触新品的机会，并以优惠政策吸引用户参与测试和反馈。'),
 Document(metadata={'doc_id': '5affe8c7-3a40-4a4c-bc07-da76132c0c45'}, page_content='本文档介绍了在VMALL平台上进行设备回收的具体步骤，包括选择要回收的物品、价格评估、查看估价结果以及提交订单的过程。\n\n一、用户首先需要通过点击VMALL网站上的“以旧换新”标志进入相应的页面，在这里可以选择想要回收的商品类别和品牌。如果找不到自己的品牌或型号，则可以通过点击加号来查找所有支持的品牌和型号。\n\n二、选择好商品后，系统会引导用户进行价格评估。在这个过程中，用户需要提供关于设备的详细信息，包括网络制式、存储容量等基本信息以及外观状况等，并最终通过“免费询价”按钮提交这些数据以获取估价结果。\n\n三、在收到估价反馈之后，如果对结果满意，则可以继续下一步操作；如果有任何疑问或希望调整评估参数，可以选择重新询价或者更换机型。确认无误后点击“换钱”，进入订单页面完成回收流程的最后一步。\n\n四、提交订单时，用户可以从两种不同的回收方式中选择一种：顺丰快递上门取货或是质检师直接到访进行检测和回收。每种方法都有其特定的操作步骤，包括提供必要的个人信息以及预约合适的时间等。\n\n五、对于在使用过程中遇到的问题或需要进一步了解的信息，可以访问专门的“以旧换新答疑”页面获取帮助和支持。'),
 Docume

## 假设性问题

假设性问题通常是每个文档都适合回答的问题。在这里，我们展示如何创建假设性问题，然后嵌入它们。

LLM可以生成针对特定问题的假设性问题列表，然后嵌入这些问题。

In [22]:
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_doc = child_text_splitter.split_documents([doc])
    for _doc in _sub_doc:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_doc)

sub_docs

Created a chunk of size 224, which is longer than the specified 100
Created a chunk of size 117, which is longer than the specified 100
Created a chunk of size 176, which is longer than the specified 100
Created a chunk of size 120, which is longer than the specified 100
Created a chunk of size 319, which is longer than the specified 100
Created a chunk of size 123, which is longer than the specified 100
Created a chunk of size 135, which is longer than the specified 100
Created a chunk of size 115, which is longer than the specified 100
Created a chunk of size 119, which is longer than the specified 100
Created a chunk of size 147, which is longer than the specified 100
Created a chunk of size 124, which is longer than the specified 100
Created a chunk of size 104, which is longer than the specified 100
Created a chunk of size 103, which is longer than the specified 100
Created a chunk of size 163, which is longer than the specified 100
Created a chunk of size 108, which is longer tha

[Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-4359.txt', 'doc_id': '0a8c93ce-9cae-46ee-a679-e2dda18101dd'}, page_content='一、什么是0分期利息\n\n您好，“0分期利息”是指买家使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息的功能，分期利息由华为商城承担。'),
 Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-4359.txt', 'doc_id': '0a8c93ce-9cae-46ee-a679-e2dda18101dd'}, page_content='注：自2023年起，商城将相关宣传将“免息”调整为“0分期利息”，主要基于中国银保监会、中国人民银行《关于进一步促进信用卡业务规范健康发展的通知》（银保监规〔2022〕13号），要求“银行业金融机构应当在分期业务合同（协议）首页和业务办理页面以明显方式展示分期业务可能产生的所有息费项目、年化利率水平和息费计算方式。向客户展示分期业务收取的资金使用成本时，应当统一采用利息形式，并明确相应的计息规则，不得采用手续费等形式，法律法规另有规定的除外。”'),
 Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-4359.txt', 'doc_id': '0a8c93ce-9cae-46ee-a679-e2dda18101dd'}, page_content='二、可以参与0分期利息活动的商品\n\n商城目前仅支持部分单品参与0分期利息，若多商品（含不支持0分期利息）合并支付则不支持0分期利息，以支付页面为准，后续会逐渐开放更多商品，敬请关注。'),
 Document(metadata={'source': '/mnt/f/kk_datasets/txt/faq-4359.txt', 'doc_id': '0a8c93ce-9cae-46ee-a679-e2dda18101dd'}, page_content='三、确认订单分期成功\n\n订单提交成功后在支付方式页面选择分期支付，点选显示0分期利息的支付方式及具体

In [23]:
from langchain_core.output_parsers import JsonOutputParser

template = """
```
{doc}
```

根据上面的文档，生成3个相关问题和答案。响应以json列表的结构返回。返回的结构参考如下:
```
[
   {{"question": "问题1", "answer": "答案1"}},
   {{"question": "问题2", "answer": "答案2"}},
   {{"question": "问题3", "answer": "答案3"}}
]
```
"""

prompt = ChatPromptTemplate.from_template(template)


chain = (
    {"doc": lambda x: x.page_content}
    | prompt
    | model
    | JsonOutputParser()
)


In [24]:
hypothetical_questions = chain.batch(sub_docs, {"max_concurrency": 5})

hypothetical_questions


[[{'question': '什么是0分期利息？',
   'answer': '“0分期利息”是指买家在使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息，这些利息由华为商城承担。'},
  {'question': '哪些支付工具可以享受0分期利息的优惠？',
   'answer': '可以享受0分期利息优惠的支付工具有花呗、招行掌上生活、工行信用卡以及银联信用卡等。'},
  {'question': '谁来承担0分期利息中的利息费用？',
   'answer': '在“0分期利息”活动中，华为商城承担了买家使用分期付款时产生的利息费用。'}],
 [{'question': '自哪一年起，商城将‘免息’宣传调整为‘0分期利息’？',
   'answer': '自2023年起，商城将相关宣传从‘免息’调整为‘0分期利息’。'},
  {'question': '为什么商城要将‘免息’宣传调整为‘0分期利息’？',
   'answer': '主要基于中国银保监会、中国人民银行的通知要求，需要明确展示分期业务可能产生的所有息费项目及计息规则，不得采用手续费等形式。'},
  {'question': '根据哪份文件，商城需要在分期业务合同首页和办理页面上以明显方式展示相关息费信息？',
   'answer': '根据中国银保监会、中国人民银行《关于进一步促进信用卡业务规范健康发展的通知》（银保监规〔2022〕13号），要求明确展示分期业务的息费项目及计息规则。'}],
 [{'question': '哪些商品可以参与0分期利息活动？',
   'answer': '目前只有部分单品支持参与0分期利息活动，具体以支付页面显示为准。'},
  {'question': '如果我购买多个商品，是否还能享受0分期利息优惠？',
   'answer': '不可以，若合并支付的商品中包含不支持0分期利息的物品，则整个订单将无法享受0分期利息优惠。'},
  {'question': '未来是否会增加更多可以参与0分期利息活动的商品？',
   'answer': '是的，商城计划逐渐开放更多的商品参与到0分期利息活动中来，请持续关注相关更新。'}],
 [{'question': '如何在支付方式页面选择分期支付？',
   'ans

In [25]:
documents = []
for item in hypothetical_questions:
    for obj in item:
        content = f"问题: {obj['question']}\n答案: {obj['answer']}"
        documents.append(Document(page_content=content, metadata={id_key: doc_ids[i]}))
        
documents

[Document(metadata={'doc_id': 'a2923e1e-56de-450d-84c8-075a903473cc'}, page_content='问题: 什么是0分期利息？\n答案: “0分期利息”是指买家在使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息，这些利息由华为商城承担。'),
 Document(metadata={'doc_id': 'a2923e1e-56de-450d-84c8-075a903473cc'}, page_content='问题: 哪些支付工具可以享受0分期利息的优惠？\n答案: 可以享受0分期利息优惠的支付工具有花呗、招行掌上生活、工行信用卡以及银联信用卡等。'),
 Document(metadata={'doc_id': 'a2923e1e-56de-450d-84c8-075a903473cc'}, page_content='问题: 谁来承担0分期利息中的利息费用？\n答案: 在“0分期利息”活动中，华为商城承担了买家使用分期付款时产生的利息费用。'),
 Document(metadata={'doc_id': 'a2923e1e-56de-450d-84c8-075a903473cc'}, page_content='问题: 自哪一年起，商城将‘免息’宣传调整为‘0分期利息’？\n答案: 自2023年起，商城将相关宣传从‘免息’调整为‘0分期利息’。'),
 Document(metadata={'doc_id': 'a2923e1e-56de-450d-84c8-075a903473cc'}, page_content='问题: 为什么商城要将‘免息’宣传调整为‘0分期利息’？\n答案: 主要基于中国银保监会、中国人民银行的通知要求，需要明确展示分期业务可能产生的所有息费项目及计息规则，不得采用手续费等形式。'),
 Document(metadata={'doc_id': 'a2923e1e-56de-450d-84c8-075a903473cc'}, page_content='问题: 根据哪份文件，商城需要在分期业务合同首页和办理页面上以明显方式展示相关息费信息？\n答案: 根据中国银保监会、中国人民银行《关于进一步促进信用卡业务规范健康发展的通知》（银保监规

In [26]:
vectorstore = Chroma(collection_name="Qustions", embedding_function=embeddings, persist_directory="/home/libing/kk_Projects/kk_langchain_base/database/vector_store")
store = InMemoryStore()
id_key = "doc_id"
retriever = MultiVectorRetriever(vectorstore=vectorstore, byte_store=store, id_key=id_key)

retriever.vectorstore.add_documents(documents)

docs_ids = [str(uuid.uuid4()) for _ in documents]

In [27]:
retriever.vectorstore.add_documents(documents)

['88cf2613-dc08-410c-bdec-216b12b57d67',
 '69013961-422d-474c-947e-3136a5fbbb6b',
 'de5b1599-2011-4d14-b73d-110a81674bcd',
 'b33b6db3-4c2f-4bbd-86b4-530fc91491cf',
 'ed761ff9-c0b5-4133-be6a-75b51af9f547',
 '60ae6310-52eb-40ee-91bf-de2bfb6f86e6',
 'a1548e62-ff34-4db0-af4a-77b261bb0f10',
 '042c1aba-3943-4554-b96c-2b079fbdc17f',
 '174bb52e-8abb-4c8b-b056-7808e39fce6e',
 '1e913920-ca36-4e19-a083-fc8e662e54af',
 'fef58d74-bc0a-4c19-9a7f-71c92fadbd5d',
 '42a2320d-c498-4cae-a1da-98d2478c27e7',
 '619f4237-ff98-4ceb-835c-294a0e5af18c',
 'e461b11c-5a97-491c-87a5-5550c62f8a56',
 'd8cfbe45-51ec-46df-8db6-01b059848ed3',
 '3a9a292e-f153-4c90-b00f-09c35bcb803e',
 'eeda8fe2-c7b4-4947-8518-5838e6553ad2',
 '514be1fe-b4fe-4fd8-9f8b-0179cdbc4637',
 '26b64b90-bd83-4f39-b4b8-59f92727fc65',
 '5f59b5e2-b9ed-4a07-a218-e7ee7b3c1c3f',
 'ef4e8cdd-1659-4560-a7aa-63c4aa32dc02',
 '7d5d9b06-d9dc-40d3-bda9-ed755ff85552',
 'cf8f9452-6f48-46d6-9957-5cd7abf56411',
 '452c9ad1-a052-4e53-9fba-a20842991ce5',
 'cb9b84e5-5ccb-

In [29]:
retriever.vectorstore.similarity_search("LangGraph涉及那些技术？")[0]

Document(metadata={'doc_id': 'a2923e1e-56de-450d-84c8-075a903473cc'}, page_content='问题: LangGraph 结合了哪些技术的优势？\n答案: LangGraph 结合了语言模型和知识图谱的优势。')